In [1]:
import xarray as xr
import numpy as np
import json

from pandas import to_datetime, IntervalIndex
from rasterio.features import geometry_mask
from rasterio.transform import from_bounds
from shapely.geometry import shape
from pathlib import Path


from util import date_interval_endpoints as endpoints
from datetime import datetime
from dateutil.rrule import rrule, MONTHLY 

home = str(Path.home())

In [2]:
cube = xr.open_zarr(home+"/stacks/13/2002/8920.zarr", chunks=None)["data"]
spatial_shape = cube.shape[-2:]

metadata = json.loads(cube.json)

# Masking
masks = []
for sid in cube['slice_ids'].data:
    m = metadata[sid]
    
    transform = from_bounds(*shape(m['footprint']).bounds, *spatial_shape)
    if m['cloud_polys']:
        mask = geometry_mask(m['cloud_polys'], out_shape = spatial_shape, transform=transform)
    else:
        mask = np.ones(spatial_shape, dtype=bool)
    mask = np.tile(mask, (len(cube.bands),1,1)) # Clone array to mask all bands at once
    masks.append(mask)

masks = np.array(masks)
cube = cube.where(masks)

# Tidy up cube
timestamps = [to_datetime(v['timestamp']) for v in metadata.values()]
cube.coords['slice_ids'] = timestamps
cube = cube.rename({'slice_ids': 'timestamps'})
cube = cube.sortby('timestamps')

# get rid of resolution in band name
cube = cube.assign_coords(bands=[b.split('_')[0] for b in cube.bands.data])

# Change bands coordinate to data variables in a Dataset
cube = xr.Dataset({b: cube.sel(dict(bands=b)).reset_coords('bands', drop=True) for b in cube.bands.data})
cube

<xarray.Dataset>
Dimensions:     (timestamps: 35, x: 256, y: 256)
Coordinates:
  * timestamps  (timestamps) datetime64[ns] 2019-06-02T09:58:14 ... 2019-08-29T10:07:49
Dimensions without coordinates: x, y
Data variables:
    B04         (timestamps, x, y) float64 291.0 296.0 294.0 ... nan nan nan
    B03         (timestamps, x, y) float64 621.0 646.0 672.0 ... nan nan nan
    B02         (timestamps, x, y) float64 390.0 394.0 396.0 ... nan nan nan
    B08         (timestamps, x, y) float64 5.122e+03 5.304e+03 ... nan nan

In [3]:
starttime = datetime(*min(timestamps).timetuple()[:3],0,0,0)
endtime = datetime(*max(timestamps).timetuple()[:3],0,0,0)

eps = endpoints(starttime, endtime, 16)
idx = IntervalIndex.from_arrays(eps[::2], eps[1::2])
idx

IntervalIndex([(2019-06-02, 2019-06-15 23:59:59], (2019-06-16, 2019-06-30 23:59:59], (2019-07-01, 2019-07-15 23:59:59], (2019-07-16, 2019-07-31 23:59:59], (2019-08-01, 2019-08-15 23:59:59], (2019-08-16, 2019-08-29 23:59:59]],
              closed='right',
              dtype='interval[datetime64[ns]]')

In [4]:
# Calculate values in time intervals
avg_cube = cube.groupby_bins('timestamps',bins=idx).mean('timestamps')

# Fake data
from numpy.random import random_sample
for b in ['B05', 'B06', 'B07', 'B8A', 'B11', 'B12']:
    avg_cube[b] = avg_cube.B04
    avg_cube[b].data = random_sample(avg_cube.B04.shape)*4000

avg_cube

<xarray.Dataset>
Dimensions:          (timestamps_bins: 6, x: 256, y: 256)
Coordinates:
  * timestamps_bins  (timestamps_bins) object (2019-06-02, 2019-06-15 23:59:59] ... (2019-08-16, 2019-08-29 23:59:59]
Dimensions without coordinates: x, y
Data variables:
    B04              (timestamps_bins, x, y) float64 324.0 329.0 ... 361.5 282.5
    B03              (timestamps_bins, x, y) float64 582.2 604.2 ... 541.5 469.5
    B02              (timestamps_bins, x, y) float64 367.2 374.2 ... 324.5 289.0
    B08              (timestamps_bins, x, y) float64 4.623e+03 ... 2.704e+03
    B05              (timestamps_bins, x, y) float64 46.74 3.413e+03 ... 658.4
    B06              (timestamps_bins, x, y) float64 3.292e+03 82.56 ... 87.42
    B07              (timestamps_bins, x, y) float64 917.3 ... 2.667e+03
    B8A              (timestamps_bins, x, y) float64 1.949e+03 ... 3.117e+03
    B11              (timestamps_bins, x, y) float64 3.202e+03 ... 1.902e+03
    B12              (timestamps_bins, x, y) float64 3.241e+03 ... 2.556e+03

In [5]:
def calculate_index(a,b):
    return ((a-b)/(a+b)+1)/2 # stretch [-1,+1] to [0,1]

ics = { # index components
  "NDVI":  ["B08", "B04"],
  "GNDVI": ["B08", "B03"],
  "BNDVI": ["B08", "B02"],
  "NDSI":  ["B11", "B12"],
  "NDWI":  ["B03", "B08"]
}

for idx in ics:
    ic1, ic2 = ics[idx]
    if ic1 in avg_cube and ic2 in avg_cube:
        avg_cube[idx] = calculate_index(avg_cube[ic1], avg_cube[ic2])*65335 # 2**16

# Typing conforming to SH Mass output
avg_cube = avg_cube.astype(np.uint16)

# CVI calculation
if "B03" in avg_cube and "B05" in avg_cube and "B08" in avg_cube:
    avg_cube['CVI'] = (avg_cube.B08 * avg_cube.B05 / avg_cube.B03**2).astype(np.float32)

avg_cube

<xarray.Dataset>
Dimensions:          (timestamps_bins: 6, x: 256, y: 256)
Coordinates:
  * timestamps_bins  (timestamps_bins) object (2019-06-02, 2019-06-15 23:59:59] ... (2019-08-16, 2019-08-29 23:59:59]
Dimensions without coordinates: x, y
Data variables:
    B04              (timestamps_bins, x, y) uint16 324 329 325 ... 412 361 282
    B03              (timestamps_bins, x, y) uint16 582 604 612 ... 583 541 469
    B02              (timestamps_bins, x, y) uint16 367 374 373 ... 359 324 289
    B08              (timestamps_bins, x, y) uint16 4623 4802 4878 ... 2838 2704
    B05              (timestamps_bins, x, y) uint16 46 3412 595 ... 56 3366 658
    B06              (timestamps_bins, x, y) uint16 3292 82 1029 ... 3787 87
    B07              (timestamps_bins, x, y) uint16 917 3286 2781 ... 1420 2666
    B8A              (timestamps_bins, x, y) uint16 1948 520 252 ... 2980 3116
    B11              (timestamps_bins, x, y) uint16 3201 1362 3254 ... 3736 1901
    B12              (timestamps_bins, x, y) uint16 3241 3698 1121 ... 669 2556
    NDVI             (timestamps_bins, x, y) uint16 61055 61145 ... 57953 59155
    GNDVI            (timestamps_bins, x, y) uint16 58026 58032 ... 54866 55670
    BNDVI            (timestamps_bins, x, y) uint16 60526 60611 ... 58631 59027
    NDSI             (timestamps_bins, x, y) uint16 32465 17588 ... 55410 27872
    NDWI             (timestamps_bins, x, y) uint16 7308 7302 ... 10468 9664
    CVI              (timestamps_bins, x, y) float32 1.4532778 ... 0.41793346